### Make sure you have run the file named 001-MAC2IP.ipynb before this file. This file needs the "Device_Namer.pickle" file generated by that file.
### You will need the [CICFlowMeter](https://www.unb.ca/cic/research/applications.html) when using this file. Make sure it is installed on your computer.

####  import related modules

In [1]:
import warnings
import pickle

warnings.filterwarnings("ignore")
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import math

#### File finder

In [2]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




#### Folder creator

In [3]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

#### Delete unnecessary files

In [4]:
def killthemall():
    them=find_the_way("./temp/",".csv")
    for t in them:
        try:
            os.remove(t)
        except:
            print(f"error about delete {t} file")

In [5]:
# find pcap file in path folder
path="./pcaps"
pcaps=find_the_way(path,".pcap")
pcaps

['./pcaps\\16-09-29.pcap',
 './pcaps\\16-10-03.pcap',
 './pcaps\\16-11-18.pcap',
 './pcaps\\16-11-22.pcap',
 './pcaps\\18-06-11.pcap',
 './pcaps\\18-06-14.pcap',
 './pcaps\\18-10-13.pcap',
 './pcaps\\18-10-16.pcap']

### Use CICFlowMeter for Feature Extraction

<img src="./img/1.jpg" alt="drawing" width="600"/>

### please watch the sample application Video (/img/video.mp4)

In [6]:
# find pcap file in path folder
path="csvs"
csvs=find_the_way(path,".csv")
csvs

['csvs\\16-09-29.pcap_Flow.csv',
 'csvs\\16-10-03.pcap_Flow.csv',
 'csvs\\16-11-18.pcap_Flow.csv',
 'csvs\\16-11-22.pcap_Flow.csv',
 'csvs\\18-06-11.pcap_Flow.csv',
 'csvs\\18-06-14.pcap_Flow.csv',
 'csvs\\18-10-13.pcap_Flow.csv',
 'csvs\\18-10-16.pcap_Flow.csv']

In [7]:
with open('Device_Namer.pickle', 'rb') as handle:
    Device_Namer = pickle.load(handle)
Device_Namer

{'16-09-29': {'192.168.1.106': '30:8c:fb:2f:e4:b2',
  '52.8.241.183': '14:cc:20:51:33:ea',
  '192.168.1.1': '14:cc:20:51:33:ea',
  '195.154.172.147': '14:cc:20:51:33:ea',
  '52.87.241.159': '14:cc:20:51:33:ea',
  '15.72.162.53': '14:cc:20:51:33:ea',
  '89.30.121.23': '14:cc:20:51:33:ea',
  '52.1.40.122': '14:cc:20:51:33:ea',
  '195.154.50.167': '14:cc:20:51:33:ea',
  '52.203.224.99': '14:cc:20:51:33:ea',
  '52.8.89.254': '14:cc:20:51:33:ea',
  '138.236.128.36': '14:cc:20:51:33:ea',
  '120.24.76.167': '14:cc:20:51:33:ea',
  '115.29.178.199': '14:cc:20:51:33:ea',
  '176.32.98.204': '14:cc:20:51:33:ea',
  '174.129.217.97': '14:cc:20:51:33:ea',
  '54.254.250.149': '14:cc:20:51:33:ea',
  '132.163.4.103': '14:cc:20:51:33:ea',
  '46.105.38.79': '14:cc:20:51:33:ea',
  '146.148.44.137': '14:cc:20:51:33:ea',
  '50.7.114.59': '14:cc:20:51:33:ea',
  '50.7.176.18': '14:cc:20:51:33:ea',
  '52.76.51.215': '14:cc:20:51:33:ea',
  '149.171.37.162': '14:cc:20:51:33:ea',
  '52.74.225.231': '14:cc:20:51:33

In [8]:
# SELECTED DEVICES
macs={'d0:52:a8:00:67:5e': 'Smart Things',
 '44:65:0d:56:cc:d3': 'Amazon Echo',
 '70:ee:50:18:34:43': 'Netatmo Welcome',
 'f4:f2:6d:93:51:f1': 'TP-Link Day Night Cloud camera',
 '00:16:6c:ab:6b:88': 'Samsung SmartCam',
 '30:8c:fb:2f:e4:b2': 'Dropcam',
 'ec:1a:59:79:f4:89': 'Belkin Wemo switch',
 '50:c7:bf:00:56:39': 'TP-Link Smart plug',
 '74:c6:3b:29:d7:1d': 'iHome',
 'ec:1a:59:83:28:11': 'Belkin wemo motion sensor',
 '18:b4:30:25:be:e4': 'NEST Protect smoke alarm',
 '70:ee:50:03:b8:ac': 'Netatmo weather station',
 'd0:73:d5:01:83:08': 'Light Bulbs LiFX Smart Bulb',
 '18:b7:9e:02:20:44': 'Triby Speaker',
 'e0:76:d0:33:bb:85': 'PIX-STAR Photo-frame',
 '70:5a:0f:e4:9b:c0': 'HP Printer',
 '08:21:ef:3b:fc:e3': 'Samsung Galaxy Tab',
 'ac:bc:32:d4:6f:2f': 'MacBook',
 '14:cc:20:51:33:ea': 'TPLink Router Bridge LAN',
 '00:17:88:2b:9a:25': 'Phillip Hue Lightbulb',
 #'7c:70:bc:5d:5e:dc': 'Canary Camera',
 '70:88:6b:10:0f:c6': 'Awair air quality monitor'}


#### Labels flows using source IP addresses and MAC address list

In [9]:
for i in csvs:
    name=i[5:-14]
    name=Device_Namer[name]
    df=pd.read_csv(i)
    label=df['Src IP'].replace(name)
    df["Label"]=label
    df.to_csv(i,index=False)
for i in csvs:
    df=pd.read_csv(i)
    label=df['Label'].replace(macs)
    df["Label"]=label
    df.to_csv(i,index=False)



In [10]:
for f in csvs:
    output=f.replace(".pcap_Flow.csv",".csv")
    os.rename(f,output)

#### Obtains sessions by merging two different files. This is done because not every file contains a sufficient number of device samples.
#### see packet numbers by sessions via [this link]( https://github.com/kahramankostas/GeMoDI/blob/main/FE/UNSW_Intersected_packetlist.xlsx) 

In [11]:
them={"DI-S1": ['./csvs/16-09-29.csv', './csvs/16-11-18.csv'],
"DI-S2": ['./csvs/16-10-03.csv','./csvs/16-11-22.csv'],
"AD-S1": ['./csvs/18-06-14.csv',  './csvs/18-10-13.csv'],
"AD-S2": [ './csvs/18-06-11.csv','./csvs/18-10-16.csv']}

In [13]:
for output in them:
    
    ths = open(f"./csvs/{output}.csv", "w")
    flag=1
    for t in them[output]:
        with open(t, "r") as file:
            while True:
                line=file.readline()
                if line=="":break
                if not line.startswith("Flow"):
                    #print(line)
                    ths.write(line)
                    #print(line)
                    #break
                else:
                    if flag:
                        ths.write(line)
                        flag=0
        os.remove(t)
    ths.close()

In [14]:
macrenamer={'d0:52:a8:00:67:5e': 'Smart Things',
 '44:65:0d:56:cc:d3': 'Amazon Echo',
 '70:ee:50:18:34:43': 'Netatmo Welcome',
 'f4:f2:6d:93:51:f1': 'TP-Link Day Night Cloud camera',
 '00:16:6c:ab:6b:88': 'Samsung SmartCam',
 '30:8c:fb:2f:e4:b2': 'Dropcam',
 'ec:1a:59:79:f4:89': 'Belkin Wemo switch',
 '50:c7:bf:00:56:39': 'TP-Link Smart plug',
 '74:c6:3b:29:d7:1d': 'iHome',
 'ec:1a:59:83:28:11': 'Belkin wemo motion sensor',
 '18:b4:30:25:be:e4': 'NEST Protect smoke alarm',
 '70:ee:50:03:b8:ac': 'Netatmo weather station',
 'd0:73:d5:01:83:08': 'Light Bulbs LiFX Smart Bulb',
 '18:b7:9e:02:20:44': 'Triby Speaker',
 'e0:76:d0:33:bb:85': 'PIX-STAR Photo-frame',
 '70:5a:0f:e4:9b:c0': 'HP Printer',
 '08:21:ef:3b:fc:e3': 'Samsung Galaxy Tab',
 'ac:bc:32:d4:6f:2f': 'MacBook',
 '14:cc:20:51:33:ea': 'TPLink Router Bridge LAN',
 '00:17:88:2b:9a:25': 'Phillip Hue Lightbulb',
# '7c:70:bc:5d:5e:dc': 'Canary Camera',
 '70:88:6b:10:0f:c6': 'Awair air quality monitor'}

devices=list(macrenamer.values())
devices

['Smart Things',
 'Amazon Echo',
 'Netatmo Welcome',
 'TP-Link Day Night Cloud camera',
 'Samsung SmartCam',
 'Dropcam',
 'Belkin Wemo switch',
 'TP-Link Smart plug',
 'iHome',
 'Belkin wemo motion sensor',
 'NEST Protect smoke alarm',
 'Netatmo weather station',
 'Light Bulbs LiFX Smart Bulb',
 'Triby Speaker',
 'PIX-STAR Photo-frame',
 'HP Printer',
 'Samsung Galaxy Tab',
 'MacBook',
 'TPLink Router Bridge LAN',
 'Phillip Hue Lightbulb',
 'Awair air quality monitor']

#### Unknown MAC/IP addresses (Labels) are removed

In [15]:
them=find_the_way(path,".csv")
them

['csvs\\AD-S1.csv', 'csvs\\AD-S2.csv', 'csvs\\DI-S1.csv', 'csvs\\DI-S2.csv']

In [16]:
proError=[]
for t in them:
    #main_labels="dns.time,http.time,icmp.data_time_relative,icmp.resptime,stun.time,tcp.ack,tcp.ack_raw,tcp.analysis.ack_rtt,tcp.analysis.initial_rtt,tcp.analysis.rto,tcp.nxtseq,tcp.options.timestamp.tsecr,tcp.options.timestamp.tsval,tcp.seq,tcp.seq_raw,tcp.time_delta,tcp.time_relative,udp.time_delta,udp.time_relative,arp.hw.size,arp.hw.type,arp.isannouncement,arp.isgratuitous,arp.isprobe,arp.opcode,arp.proto.size,arp.seconds-since-duplicate-address-frame,dhcp.flags.bc,dhcp.hops,dhcp.hw.addr_padding,dhcp.hw.len,dhcp.option.dhcp,dhcp.option.dhcp_max_message_size,dhcp.option.end,dhcp.option.interface_mtu,dhcp.option.ip_address_lease_time,dhcp.option.padding,dhcp.option.rebinding_time_value,dhcp.option.renewal_time_value,dhcp.secs,dhcp.type,dns.count.add_rr,dns.count.answers,dns.count.auth_rr,dns.count.queries,dns.extraneous,dns.extraneous.data,dns.extraneous.length,dns.flags.authenticated,dns.flags.authoritative,dns.flags.checkdisable,dns.flags.opcode,dns.flags.rcode,dns.flags.recavail,dns.flags.recdesired,dns.flags.response,dns.flags.truncated,dns.flags.z,dns.response_to,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,dns.retransmit_response,dns.retransmit_response_in,dns.soa.expire_limit,dns.soa.minimum_ttl,dns.soa.refresh_interval,dns.soa.retry_interval,dns.soa.serial_number,dns.unsolicited,eapol.keydes.key_iv,eapol.keydes.key_len,eapol.keydes.replay_counter,eapol.keydes.type,eapol.len,eapol.type,eapol.version,eth.dst.ig,eth.dst.lg,eth.dst.oui,eth.fcs.status,eth.len,eth.padding,eth.padding_bad,eth.src.ig,eth.src.lg,eth.src.oui,http.chat,http.content_length,http.content_length_header,http.notification,http.prev_request_in,http.prev_response_in,http.request,http.request_in,http.request_number,http.response,http.response.code,http.response_number,icmp.checksum.status,icmp.code,icmp.ident,icmp.ident_le,icmp.resp_to,icmp.seq,icmp.seq_le,icmp.type,icmp.unused,icmpv6.checksum.status,icmpv6.code,icmpv6.mldr.nb_mcast_records,icmpv6.opt,icmpv6.opt.length,icmpv6.opt.type,icmpv6.reserved,icmpv6.type,igmp.checksum.status,igmp.max_resp,igmp.num_grp_recs,igmp.version,ip.bogus_ip_length,ip.len,ip.opt.len,ip.opt.ra,ip.opt.type,ip.opt.type.class,ip.opt.type.copy,ip.opt.type.number,ntp.flags.li,ntp.flags.mode,ntp.flags.vn,ntp.ppoll,ntp.precision,ntp.rootdelay,ntp.rootdispersion,ntp.stratum,stun.att.crc32.status,stun.att.error,stun.att.error.class,stun.att.lifetime,stun.attributes,stun.length,stun.network_version,stun.response-to,tcp.analysis,tcp.analysis.ack_lost_segment,tcp.analysis.acks_frame,tcp.analysis.bytes_in_flight,tcp.analysis.duplicate_ack_frame,tcp.analysis.duplicate_ack_num,tcp.analysis.flags,tcp.analysis.keep_alive,tcp.analysis.keep_alive_ack,tcp.analysis.lost_segment,tcp.analysis.out_of_order,tcp.analysis.push_bytes_sent,tcp.analysis.retransmission,tcp.analysis.reused_ports,tcp.analysis.rto_frame,tcp.analysis.spurious_retransmission,tcp.analysis.window_update,tcp.checksum.status,tcp.completeness,tcp.connection.fin,tcp.connection.fin_active,tcp.connection.fin_passive,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.dstport,tcp.fin_retransmission,tcp.flags.ack,tcp.flags.ae,tcp.flags.cwr,tcp.flags.ece,tcp.flags.fin,tcp.flags.push,tcp.flags.res,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.hdr_len,tcp.len,tcp.options.mss_val,tcp.options.sack.count,tcp.options.sack.dsack,tcp.options.sack.dsack_le,tcp.options.sack.dsack_re,tcp.options.sack_le,tcp.options.sack_perm,tcp.options.sack_re,tcp.options.wscale.multiplier,tcp.options.wscale.shift,tcp.pdu.size,tcp.reassembled.length,tcp.segment.count,tcp.segment.overlap,tcp.segments,tcp.srcport,tcp.stream,tcp.urgent_pointer,tcp.window_size,tcp.window_size_scalefactor,tcp.window_size_value,tls.alert_message,tls.change_cipher_spec,tls.handshake.certificates,tls.handshake.certificates_length,tls.handshake.challenge,tls.handshake.challenge_length,tls.handshake.cipher_spec_len,tls.handshake.cipher_suites_length,tls.handshake.ciphersuites,tls.handshake.client_point_len,tls.handshake.comp_method,tls.handshake.comp_methods,tls.handshake.comp_methods_length,tls.handshake.epms_len,tls.handshake.extension.heartbeat.mode,tls.handshake.extensions_ec_point_formats,tls.handshake.extensions_ec_point_formats_length,tls.handshake.extensions_length,tls.handshake.extensions_reneg_info_len,tls.handshake.extensions_server_name_len,tls.handshake.extensions_server_name_list_len,tls.handshake.extensions_server_name_type,tls.handshake.extensions_supported_groups,tls.handshake.extensions_supported_groups_length,tls.handshake.g_len,tls.handshake.p_len,tls.handshake.server_point_len,tls.handshake.session_id_length,tls.handshake.session_ticket_length,tls.handshake.session_ticket_lifetime_hint,tls.handshake.sig_hash_alg_len,tls.handshake.sig_hash_algs,tls.handshake.sig_len,tls.handshake.yc_len,tls.handshake.ys_len,tls.resumed,tls.sct.scts_length,udp.checksum.status,udp.dstport,udp.length,udp.length.bad,udp.possible_traceroute,udp.srcport,udp.stream,eth.dst,eth.src,stun.cookie,stun.id,stun.value,tcp.options,tcp.options.mss,tcp.options.wscale,ntp.refid,arp.proto.type,dhcp.flags,dhcp.flags.reserved,dhcp.id,dns.flags,dns.id,eth.fcs,eth.type,icmp.checksum,icmpv6.checksum,igmp.checksum,igmp.type,ip.id,ipv6.flow,ipv6.tclass,ntp.flags,stun.att.crc32,stun.att.transp,stun.type,stun.type.class,stun.type.method,stun.type.method-assignment,tcp.checksum,tcp.flags,tls.handshake.server_curve_type,tls.handshake.server_named_curve,tls.handshake.version,udp.checksum,http.connection,http.request.method,http.response.code.desc,http.response.phrase,tcp.flags.str,_ws.col.Protocol,tls.handshake.sig_hash_alg,dhcp.hw.type,ip.dsfield,ip.flags,ipv6.opt.type,ipv6.opt.type.rest,stun.att.family,stun.att.ipv4-xord,stun.att.port-xord,tcp.segment,arp.duplicate-address-detected,arp.duplicate-address-frame,dns.srv.port,dns.srv.priority,dns.srv.weight,eth.addr.oui,eth.ig,eth.lg,ip.checksum.status,ip.dsfield.dscp,ip.dsfield.ecn,ip.flags.df,ip.flags.mf,ip.flags.rb,ip.frag_offset,ip.hdr_len,ip.proto,ip.ttl,ip.version,ipv6.opt,ipv6.opt.length,ipv6.opt.type.action,ipv6.opt.type.change,stun.att.padding,stun.att.port,stun.att.reserved,tcp.analysis.duplicate_ack,tls.record.length,Payload_Entropy,dstport,srcport,dstport_class,srcport_class,Label\n"
    ths = open(t.replace(".csv","_.csv"), "w")
    #ths.write(main_labels)

    with open(t, "r") as file:
        line=file.readline()
        ths.write(line)
        while True:
                line=file.readline()
                if line=="":break
                temp=line[:-1]
                temp=temp.split(",")
                if temp[-1] in devices:
                        ths.write(line)
    ths.close()
    os.remove(t)


for f in them:
    print(f)
    output=f.replace(".csv","_.csv")
    os.rename(output,f)

csvs\AD-S1.csv
csvs\AD-S2.csv
csvs\DI-S1.csv
csvs\DI-S2.csv


#### Parts containing INF errors have been fixed.  Large numeric values were given instead of INF

In [17]:
inf_problem=['Bwd Byts/b Avg',
 'Bwd Pkts/b Avg',
 'Bwd Pkts/s',
 'Down/Up Ratio',
 #'Dst IP',
 'Flow Byts/s',
 #'Flow ID',
 'Flow Pkts/s',
 'Fwd Byts/b Avg',
 'Fwd Pkts/b Avg',
 'Fwd Pkts/s']

for i in them:
    print(i)
    df=pd.read_csv(i)
    for ii in inf_problem:
        df[ii]=df[ii].replace([np.inf, -np.inf], np.iinfo(np.uint32).max)  
    df.to_csv(i,index=False)




csvs\AD-S1.csv
csvs\AD-S2.csv
csvs\DI-S1.csv
csvs\DI-S2.csv


### if you want a smaller sample, try the following